In [9]:
TOMTOM_API_KEY = '28vkTeAr3JbspIjfep0rttcarUn38ZAA'

In [10]:
import requests
def get_traffic_data(lat, lon):
    url = f"https://api.tomtom.com/traffic/services/4/flowSegmentData/absolute/10/json?point={lat}%2C{lon}&key={TOMTOM_API_KEY}"
    response = requests.get(url)
    data = response.json()
    flow = data['flowSegmentData']
    return {
        "currentSpeed": flow['currentSpeed'],
        "freeFlowSpeed": flow['freeFlowSpeed'],
    }

In [14]:
def label_congestion(current_speed, free_flow_speed):
    ratio = current_speed / free_flow_speed
    if ratio > 0.9:
        return "Low"
    elif ratio > 0.6:
        return "Medium"
    else:
        return "High"

In [15]:
import pandas as pd
import time

coordinates = [
    (28.6339, 77.2090),  # Delhi - North
    (28.5939, 77.2090),  # Delhi - South
    (28.6139, 77.2290),  # Delhi - East
    (28.6139, 77.1890),  # Delhi - West
    (28.6139, 77.2090),  # Delhi - Center
    (19.0960, 72.8777),  # Mumbai - North
    (19.0560, 72.8777),  # Mumbai - South
    (19.0760, 72.8977),  # Mumbai - East
    (19.0760, 72.8577),  # Mumbai - West
    (19.0760, 72.8777),  # Mumbai - Center
    (13.1027, 80.2707),  # Chennai - North
    (13.0627, 80.2707),  # Chennai - South
    (13.0827, 80.2907),  # Chennai - East
    (13.0827, 80.2507),  # Chennai - West
    (13.0827, 80.2707),  # Chennai - Center
    (12.9916, 77.5946),  # Bangalore - North
    (12.9516, 77.5946),  # Bangalore - South
    (12.9716, 77.6146),  # Bangalore - East
    (12.9716, 77.5746),  # Bangalore - West
    (12.9716, 77.5946),  # Bangalore - Center
    (22.5926, 88.3639),  # Kolkata - North
    (22.5526, 88.3639),  # Kolkata - South
    (22.5726, 88.3839),  # Kolkata - East
    (22.5726, 88.3439),  # Kolkata - West
    (22.5726, 88.3639),  # Kolkata - Center
    (17.4050, 78.4867),  # Hyderabad - North
    (17.3650, 78.4867),  # Hyderabad - South
    (17.3850, 78.5067),  # Hyderabad - East
    (17.3850, 78.4667),  # Hyderabad - West
    (17.3850, 78.4867),  # Hyderabad - Center
    (23.0425, 72.5714),  # Ahmedabad - North
    (23.0025, 72.5714),  # Ahmedabad - South
    (23.0225, 72.5914),  # Ahmedabad - East
    (23.0225, 72.5514),  # Ahmedabad - West
    (23.0225, 72.5714),  # Ahmedabad - Center
    (26.9324, 75.7873),  # Jaipur - North
    (26.8924, 75.7873),  # Jaipur - South
    (26.9124, 75.8073),  # Jaipur - East
    (26.9124, 75.7673),  # Jaipur - West
    (26.9124, 75.7873),  # Jaipur - Center
    (18.5404, 73.8567),  # Pune - North
    (18.5004, 73.8567),  # Pune - South
    (18.5204, 73.8767),  # Pune - East
    (18.5204, 73.8367),  # Pune - West
    (18.5204, 73.8567),  # Pune - Center
    (21.1902, 72.8311),  # Surat - North
    (21.1502, 72.8311),  # Surat - South
    (21.1702, 72.8511),  # Surat - East
    (21.1702, 72.8111),  # Surat - West
    (21.1702, 72.8311),  # Surat - Center
    (11.0368, 76.9558),  # Coimbatore - North
    (10.9968, 76.9558),  # Coimbatore - South
    (11.0168, 76.9758),  # Coimbatore - East
    (11.0168, 76.9358),  # Coimbatore - West
    (11.0168, 76.9558),  # Coimbatore - Center
    (21.1658, 79.0882),  # Nagpur - North
    (21.1258, 79.0882),  # Nagpur - South
    (21.1458, 79.1082),  # Nagpur - East
    (21.1458, 79.0682),  # Nagpur - West
    (21.1458, 79.0882),  # Nagpur - Center
    (26.8667, 80.9462),  # Lucknow - North
    (26.8267, 80.9462),  # Lucknow - South
    (26.8467, 80.9662),  # Lucknow - East
    (26.8467, 80.9262),  # Lucknow - West
    (26.8467, 80.9462),  # Lucknow - Center
    (22.7396, 75.8577),  # Indore - North
    (22.6996, 75.8577),  # Indore - South
    (22.7196, 75.8777),  # Indore - East
    (22.7196, 75.8377),  # Indore - West
    (22.7196, 75.8577),  # Indore - Center
    (23.2799, 77.4126),  # Bhopal - North
    (23.2399, 77.4126),  # Bhopal - South
    (23.2599, 77.4326),  # Bhopal - East
    (23.2599, 77.3926),  # Bhopal - West
    (23.2599, 77.4126),  # Bhopal - Center
    (30.7533, 76.7794),  # Chandigarh - North
    (30.7133, 76.7794),  # Chandigarh - South
    (30.7333, 76.7994),  # Chandigarh - East
    (30.7333, 76.7594),  # Chandigarh - West
    (30.7333, 76.7794),  # Chandigarh - Center
    (25.6141, 85.1376),  # Patna - North
    (25.5741, 85.1376),  # Patna - South
    (25.5941, 85.1576),  # Patna - East
    (25.5941, 85.1176),  # Patna - West
    (25.5941, 85.1376),  # Patna - Center
    (29.9657, 78.1642),  # Haridwar - North
    (29.9257, 78.1642),  # Haridwar - South
    (29.9457, 78.1842),  # Haridwar - East
    (29.9457, 78.1442),  # Haridwar - West
    (29.9457, 78.1642),  # Haridwar - Center
    (34.1037, 74.7973),  # Srinagar - North
    (34.0637, 74.7973),  # Srinagar - South
    (34.0837, 74.8173),  # Srinagar - East
    (34.0837, 74.7773),  # Srinagar - West
    (34.0837, 74.7973),  # Srinagar - Center
    (15.3193, 74.1240),  # Goa - North
    (15.2793, 74.1240),  # Goa - South
    (15.2993, 74.1440),  # Goa - East
    (15.2993, 74.1040),  # Goa - West
    (15.2993, 74.1240),  # Goa - Center
]


rows = []

for lat, lon in coordinates:
    traffic = get_traffic_data(lat, lon)
    label = label_congestion(traffic["currentSpeed"], traffic["freeFlowSpeed"])
    rows.append([
        traffic["currentSpeed"],
        traffic["freeFlowSpeed"],
        label
    ])
    time.sleep(1)

df = pd.DataFrame(rows, columns=['currentSpeed', 'freeFlowSpeed', 'label'])
df.to_csv('traffic_data.csv', index=False)
df


,currentSpeed,freeFlowSpeed,label
0,15,27,High
1,22,36,Medium
2,39,39,Low
3,33,45,Medium
4,19,25,Medium
...,...,...,...
95,32,32,Low
96,49,49,Low
97,38,38,Low
98,49,49,Low


In [16]:
!jupyter nbconvert --to script Traffic_data_collection.ipynb


[NbConvertApp] Converting notebook Traffic_data_collection.ipynb to script
[NbConvertApp] Writing 5580 bytes to Traffic_data_collection.py
